In [6]:
import nltk
from nltk.stem.lancaster import LancasterStemmer
import string

stemmer = LancasterStemmer()
tbl = str.maketrans({key:None for key in string.punctuation})

def remove_punctuation(txt:str) -> str:
    return txt.translate(tbl)

def tokenize(txt:str):
    return nltk.word_tokenize(txt)

def stem(token):
    return [stemmer.stem(w.lower()) for w in token]

ex = "Hello world!"
print(ex)
ex = remove_punctuation(ex)
print(ex)
ex = tokenize(ex)
print(ex)
ex = stem(ex)
print(ex)

Hello world!
Hello world
['Hello', 'world']
['hello', 'world']


In [7]:
s = LancasterStemmer()

print(s.stem('seeing'))

see


In [14]:
import spacy
from spacy import displacy
nlp = spacy.load("en_core_web_sm")

doc = nlp("Hello dear Tom, how are you my friend ?")
for ent in doc.ents:
    print(ent.sentiment)

displacy.render(doc, style = "ent")

0.0


In [55]:
from nltk.stem import WordNetLemmatizer
import nltk
t = nlp("I live in the usa! We are living in the US")
lemmatizer = WordNetLemmatizer()

for token in t:
    #print(token.text," -> ", lemmatizer.lemmatize(token.text))
    if token.lemma_ not in string.punctuation:
        print(token.text," -> ", token.lemma_, " :: ")
for ent in t.ents:
    print(ent.label_)


I  ->  -PRON-  :: 
live  ->  live  :: 
in  ->  in  :: 
the  ->  the  :: 
usa  ->  usa  :: 
We  ->  -PRON-  :: 
are  ->  be  :: 
living  ->  live  :: 
in  ->  in  :: 
the  ->  the  :: 
US  ->  US  :: 
GPE


In [58]:
print(type(nlp))

<class 'spacy.lang.en.English'>


In [ ]:
from spacy.tokens.token import Token
from spacy.tokens.doc import Doc
from typing import List, Iterable
from functools import partial

def tokenize(sentence : str, spacy_model) -> Doc:
    return spacy_model(sentence)

def lemmatize(tokens:Doc, to_avoid:Iterable) -> List[str]:
    returned_tokens = []
    for token in tokens:
        if token not in to_avoid and token.lemma_ not in to_avoid:
            returned_tokens.append(token.lemma_)
    return returned_tokens

def sent_preprocess(sentence:str, spacy_model, to_avoid:Iterable) -> List[str]:
    pipe = (partial(tokenize, spacy_model=spacy_model),
            partial(lemmatize, to_avoid=to_avoid))
    x = sentence
    for f in pipe:
        x = f(x)
    return x

In [59]:
x = list("prout")
print(x)

['p', 'r', 'o', 'u', 't']
